In [1]:
from summarytools.summarytools import dfSummary
import pandas as pd
from pathlib import Path

In [2]:
data_dir = Path('./data')
data = []
for d in data_dir.glob("*.csv"):
    list_data = pd.read_csv(d)
    data += [list_data]
data = pd.concat(data)
data.head()

,中文名,英文名,曾用名,民族,国家地区,出生日期,出生地,身高,体重,血型,...,职业,经纪公司,代表作,相关明星,image_url,image_path,name,page_url,avatar_url,微博
0,王诗槐,NaN,NaN,汉族,内地,1957-11-27,安徽合肥,178CM,65KG,A型,...,演员,安徽省话剧团,《淑女之家》,NaN,https://image.ijq.tv/201502/26/10-33-25-38-12.jpg,images/10-33-25-38-12.jpg,王诗槐,https://www.ijq.tv/mingxing/1287.html,https://image.ijq.tv/201502/26/10-33-25-38-12.jpg,NaN
1,王文绮,Vicky.Q,NaN,汉族,内地,1990-2-2,西安,174CM,50KG,B型,...,演员,中盟世纪,《冰封：重生之门温柔的诱惑》《裸漂》,江铠同,https://image.ijq.tv/201502/26/10-42-42-33-8.jpg,images/10-42-42-33-8.jpg,王文绮,https://www.ijq.tv/mingxing/1288.html,https://image.ijq.tv/201502/26/10-42-42-33-8.jpg,王文绮的新浪微博
2,冯嘉怡,NaN,NaN,汉族,内地,1969-7-10,北京,180CM,78KG,AB型,...,演员,NaN,《我的左手右手》《幸福保卫战》《裸婚时代》《媳妇的美好时代》,搭档吴彦祖 搭档姚晨,https://image.ijq.tv/201502/26/10-14-20-35-10.jpg,images/10-14-20-35-10.jpg,冯嘉怡,https://www.ijq.tv/mingxing/1286.html,https://image.ijq.tv/201502/26/10-14-20-35-10.jpg,冯嘉怡的新浪微博
3,余男,Yu Nan,NaN,汉族,内地,1976-9-5,辽宁省大连市中山区,169CM,50KG,A型,...,演员,创新艺人经纪公司CAA,《敢死队2》《月蚀》《惊蛰》《图雅的婚事》,旧爱王全安 搭档王学兵 搭档梁家辉 搭档陈思诚,https://image.ijq.tv/201502/23/16-17-18-22-10.png,images/16-17-18-22-10.png,余男,https://www.ijq.tv/mingxing/1284.html,https://image.ijq.tv/201502/23/16-17-18-22-10.png,余男的新浪微博
4,于月仙,NaN,谢大脚,回族,内地,1971-4-24,内蒙古赤峰,NaN,55KG,NaN,...,演员,NaN,《乡村爱情系列》《马大帅系列》《策马啸西风》,NaN,https://image.ijq.tv/201502/20/16-34-49-41-1.jpg,images/16-34-49-41-1.jpg,于月仙,https://www.ijq.tv/mingxing/1281.html,https://image.ijq.tv/201502/20/16-34-49-41-1.jpg,NaN


In [3]:
import numpy as np
def parse_height(x):
    x = str(x).replace('CM','')
    try:
        x = int(x)
    except:
        x = np.nan
    return x

def parse_weight(x):
    x = str(x).replace('KG','')
    try:
        x = int(x)
    except:
        x = np.nan
    return x

def parse_age(x, end_date = '2020-12-31'):
    end = pd.to_datetime(end_date)
    try:
        x = pd.to_datetime(x)
        x = int((end - x).days / 365)
    except:
        x = np.nan
    return x

In [4]:
data['height'] = data['身高'].map(parse_height) / 100
data['weight'] = data['体重'].map(parse_weight)
data['bmi'] = (data['weight'] / data['height'].map(lambda i: i ** 2)).round(2)
data['dob'] = data['出生日期']
data['age'] = data['出生日期'].map(parse_age)

In [5]:
# remove duplicates
data = data.groupby(['name', 'dob'], as_index=False).apply(lambda x: x.assign(
    rank=x['image_url'].rank(method='dense', ascending=False)))
data = data.loc[data['rank'] == 1,:]

In [6]:
fields = ['name', 'dob','age', 'height','weight','bmi','page_url','image_url','image_path']
chinese_celebrity = data[fields].dropna().rename(columns=str.upper)

In [7]:
dfSummary(chinese_celebrity)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,NAME[object],1. 张航2. 吕一3. 王婷4. 赵越5. 师春玲6. 王鹏7. 王瑛瑛8. 崔航9. 赵亮10. 王骁11. other,"3 (0.1%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)4,677 (99.6%)",,0(0.0%)
2,DOB[object],1. 19942. 1987-02-223. 1982-6-284. 1987-3-295. 1988-8-216. 1995-10-87. 1983-9-38. 1990-11-299. 1961-12-0310. 1980-10-2311. other,"9 (0.2%)7 (0.1%)7 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)4 (0.1%)4 (0.1%)4 (0.1%)4,643 (98.8%)",,0(0.0%)
3,AGE[float64],Mean (sd) : 33.6 (10.8)min < med < max:5.0 < 31.0 < 120.0IQR (CV) : 12.0 (3.1),77 distinct values,,0(0.0%)
4,HEIGHT[float64],Mean (sd) : 1.7 (0.1)min < med < max:1.5 < 1.7 < 2.0IQR (CV) : 0.1 (21.2),48 distinct values,,0(0.0%)
5,WEIGHT[float64],Mean (sd) : 57.5 (13.0)min < med < max:15.0 < 56.0 < 200.0IQR (CV) : 21.0 (4.4),94 distinct values,,0(0.0%)
6,BMI[float64],Mean (sd) : 19.0 (3.0)min < med < max:6.7 < 18.7 < 65.3IQR (CV) : 4.2 (6.3),621 distinct values,,0(0.0%)
7,PAGE_URL[object],1. https://www.ijq.tv/mingxing/142. https://www.ijq.tv/mingxing/153. https://www.ijq.tv/mingxing/154. https://www.ijq.tv/mingxing/115. https://www.ijq.tv/mingxing/186. https://www.ijq.tv/mingxing/157. https://www.ijq.tv/mingxing/278. https://www.ijq.tv/mingxing/149. https://www.ijq.tv/mingxing/6110. https://www.ijq.tv/mingxing/1511. other,"1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)4,688 (99.8%)",,0(0.0%)
8,IMAGE_URL[object],1. https://image.ijq.tv/201711/022. https://image.ijq.tv/201908/143. https://image.ijq.tv/201704/294. https://image.ijq.tv/201803/265. https://image.ijq.tv/201909/056. https://image.ijq.tv/201804/207. https://image.ijq.tv/201909/248. https://image.ijq.tv/201911/279. https://image.ijq.tv/201503/2710. https://image.ijq.tv/202001/0211. other,"2 (0.0%)2 (0.0%)2 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)4,685 (99.7%)",,0(0.0%)
9,IMAGE_PATH[object],1. images/15-18-43-77-10.jpg2. images/14-38-37-35-39.jpg3. images/11-37-17-14-47.png4. images/14-20-36-59-19.png5. images/11-30-46-38-8.jpg6. images/16-34-45-44-16.png7. images/14-31-58-19-12.jpg8. images/11-52-46-88-12.jpg9. images/16-31-01-55-47.png10. images/15-46-12-81-7.jpg11. other,"2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)4,684 (99.7%)",,0(0.0%)


In [8]:
chinese_celebrity.to_csv('./chinese-celebrity-faces.csv', index = False)